In [1]:
import numpy as np
import keras
from keras.models import Sequential, Model, load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform

In [2]:
inputShape = (224, 224, 3)

In [15]:
def identity_block(X, kernel_size, filters, stage, block):
    conv_name_base = "conv" + str(stage) + block
    bn_name_base = "bn_conv" + str(stage) + block
    
    F1, F2, F3 = filters
    X_shortcut = X
    
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=kernel_size, strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [16]:
def convolutional_block(X, kernel_size, filters, stage, block, s=2):
    conv_name_base = "conv" + str(stage) + block
    bn_name_base = "bn_conv" + str(stage) + block
    
    F1, F2, F3 = filters
    X_shortcut = X
    
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=kernel_size, strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X
    

In [17]:
def resnet50(input_shape = (224, 224, 3)):
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((3, 3))(X_input)
    
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    X = convolutional_block(X, (3,3), [64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, (3,3), [64, 64, 256], stage=2, block='b')
    X = identity_block(X, (3,3), [64, 64, 256], stage=2, block='c')
    
    X = convolutional_block(X, (3,3), [128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, (3,3), [128, 128, 512], stage=3, block='b')
    X = identity_block(X, (3,3), [128, 128, 512], stage=3, block='c')
    X = identity_block(X, (3,3), [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, (3,3),[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, (3,3), [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, (3,3), [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, (3,3), [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, (3,3), [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, (3,3), [256, 256, 1024], stage=4, block='f')

    X = convolutional_block(X, (3,3), [512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, (3,3), [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, (3,3), [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [18]:
base_model = resnet50(input_shape=(224, 224, 3))

In [21]:
fcModel = base_model.output
fcModel = Flatten()(fcModel)
fcModel = Dense(256, activation='relu', name='Dense1',kernel_initializer=glorot_uniform(seed=0))(fcModel)
fcModel = Dense(128, activation='relu', name='Dense2',kernel_initializer=glorot_uniform(seed=0))(fcModel)
fcModel = Dense( 5,activation='softmax', name='Output',kernel_initializer=glorot_uniform(seed=0))(fcModel)

In [23]:
model = Model(inputs=base_model.input, outputs=fcModel)

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_3 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_4[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 64  9472        ['zero_padding2d_3[0][0]']       
                                )                                                             

 n)                                                                                               
                                                                                                  
 activation_17 (Activation)     (None, 55, 55, 64)   0           ['bn_conv2c2b[0][0]']            
                                                                                                  
 conv2c2c (Conv2D)              (None, 55, 55, 256)  16640       ['activation_17[0][0]']          
                                                                                                  
 bn_conv2c2c (BatchNormalizatio  (None, 55, 55, 256)  1024       ['conv2c2c[0][0]']               
 n)                                                                                               
                                                                                                  
 add_4 (Add)                    (None, 55, 55, 256)  0           ['bn_conv2c2c[0][0]',            
          

 bn_conv3c2c (BatchNormalizatio  (None, 28, 28, 512)  2048       ['conv3c2c[0][0]']               
 n)                                                                                               
                                                                                                  
 add_7 (Add)                    (None, 28, 28, 512)  0           ['bn_conv3c2c[0][0]',            
                                                                  'activation_24[0][0]']          
                                                                                                  
 activation_27 (Activation)     (None, 28, 28, 512)  0           ['add_7[0][0]']                  
                                                                                                  
 conv3d2a (Conv2D)              (None, 28, 28, 128)  65664       ['activation_27[0][0]']          
                                                                                                  
 bn_conv3d

                                                                                                  
 add_10 (Add)                   (None, 14, 14, 1024  0           ['bn_conv4b2c[0][0]',            
                                )                                 'activation_33[0][0]']          
                                                                                                  
 activation_36 (Activation)     (None, 14, 14, 1024  0           ['add_10[0][0]']                 
                                )                                                                 
                                                                                                  
 conv4c2a (Conv2D)              (None, 14, 14, 256)  262400      ['activation_36[0][0]']          
                                                                                                  
 bn_conv4c2a (BatchNormalizatio  (None, 14, 14, 256)  1024       ['conv4c2a[0][0]']               
 n)       

                                )                                                                 
                                                                                                  
 conv4f2a (Conv2D)              (None, 14, 14, 256)  262400      ['activation_45[0][0]']          
                                                                                                  
 bn_conv4f2a (BatchNormalizatio  (None, 14, 14, 256)  1024       ['conv4f2a[0][0]']               
 n)                                                                                               
                                                                                                  
 activation_46 (Activation)     (None, 14, 14, 256)  0           ['bn_conv4f2a[0][0]']            
                                                                                                  
 conv4f2b (Conv2D)              (None, 14, 14, 256)  590080      ['activation_46[0][0]']          
          

 bn_conv5c2a (BatchNormalizatio  (None, 7, 7, 512)   2048        ['conv5c2a[0][0]']               
 n)                                                                                               
                                                                                                  
 activation_55 (Activation)     (None, 7, 7, 512)    0           ['bn_conv5c2a[0][0]']            
                                                                                                  
 conv5c2b (Conv2D)              (None, 7, 7, 512)    2359808     ['activation_55[0][0]']          
                                                                                                  
 bn_conv5c2b (BatchNormalizatio  (None, 7, 7, 512)   2048        ['conv5c2b[0][0]']               
 n)                                                                                               
                                                                                                  
 activatio